# Using LlamaIndex to Automate the fine-tuning of GPT-3.5-turbo on source documents

Primarly Extended from [this](https://colab.research.google.com/drive/1vWeJBXdFEObuihO7Z8ui2CAYkdHQORqo?usp=sharing) notebook, we'll take a look at how we can wrap this process into Chainlit and have our own dynamic fine-tuning machine!

In [1]:
!pip install -q -U llama-index pypdf sentence-transformers ragas openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.60.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 3.20.0 which is incompatible.


In [2]:
import os
from getpass import getpass

openai_api_key = getpass("Enter your OpenAI API key: ")
os.environ["OPENAI_API_KEY"] = openai_api_key

Enter your OpenAI API key: ········


In [3]:
!curl https://jaydixit.com/files/PDFs/TheultimateHitchhikersGuide.pdf --output hitchhikers.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3322k  100 3322k    0     0   649k      0  0:00:05  0:00:05 --:--:--  830k


In [4]:
from llama_index import SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI
from llama_index.evaluation import DatasetGenerator

documents = SimpleDirectoryReader(
    input_files=["hitchhikers.pdf"]
).load_data()

# Shuffle the documents
import random

random.seed(42)
random.shuffle(documents)

gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

question_gen_query = (
    "You are a Teacher/ Professor. Your task is to setup "
    "a quiz/examination. Using the provided context from a "
    "report on climate change and the oceans, formulate "
    "a single question that captures an important fact from the "
    "context. Restrict the question to the context information provided."
)

dataset_generator = DatasetGenerator.from_documents(
    documents[:50],
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Generative Questions with `gpt-3.5-turbo`

We can use the `generate_questions_from_nodes()` method of our dataset generator to produce a number of questions that will be used to fine-tune!

> NOTE: This cell will take ~30s-2min.

In [5]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


Let's take a peek and see what was created!

In [6]:
questions[0]

'What did Zaphod find on the external monitor screens in the Horsehead Nebula?'

Now we can save our questions into a text file for later use.

In [7]:
with open("train_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Evaluation Generator

Let's generate questions from a different segment of our documents in order to build a robust test for our RAQA.

In [8]:
dataset_generator = DatasetGenerator.from_documents(
    documents[
        50:
    ],  # since we generated ~1 question for 40 documents, we can skip the first 40
    question_gen_query=question_gen_query,
    service_context=gpt_35_context,
)

Again, we'll use `gpt-3.5-turbo` to generate some questions!

In [9]:
questions = dataset_generator.generate_questions_from_nodes(num=40)
print("Generated ", len(questions), " questions")

Generated  40  questions


Now we can save our results for evaluations later!

In [10]:
with open("eval_questions.txt", "w") as f:
    for question in questions:
        f.write(question + "\n")

### Evaluating base `gpt-3.5-turbo`

We'll load up our evaluation questions and get to it!

In [11]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

This next cell is constructing our `VectorIndex` so we can move onto testing the base model.

In [12]:
from llama_index import VectorStoreIndex

# limit the context window to 2048 tokens so that refine is used
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3), context_window=2048
)

index = VectorStoreIndex.from_documents(documents, service_context=gpt_35_context)

query_engine = index.as_query_engine(similarity_top_k=2)

Here is where we're actually putting the model to the test!

In [13]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

Now that we've tested our model - let's evaluate it to see how it performed!

We're testing our model with the `ragas` framework - found [here](https://github.com/explodinggradients/ragas)

You'll notice that we're testing two primary metrics:

- [`answer_relevancy`](https://github.com/explodinggradients/ragas/blob/a55c3be8b2389501c5c761df9070126027a4d1d6/src/ragas/metrics/answer_relevance.py#L32): This measures how relevant is the generated answer to the prompt. If the generated answer is incomplete or contains redundant information the score will be low. This is quantified by working out the chance of an LLM generating the given question using the generated answer. Values range (0,1), higher the better.
- [`faithfulness`](https://github.com/explodinggradients/ragas/blob/a55c3be8b2389501c5c761df9070126027a4d1d6/src/ragas/metrics/faithfulnes.py#L63): This measures the factual consistency of the generated answer against the given context. This is done using a multi step paradigm that includes creation of statements from the generated answer followed by verifying each of these statements against the context. The answer is scaled to (0,1) range. Higher the better.

Read more about their implementations [here](https://github.com/explodinggradients/ragas/blob/main/docs/metrics.md)

Again, these cells might take some time to complete - be patient!

In [14]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

evaluating with [answer_relevancy]


100%|█████████████████████████████████████████████| 3/3 [01:45<00:00, 35.23s/it]


evaluating with [faithfulness]


100%|████████████████████████████████████████████| 3/3 [05:29<00:00, 109.89s/it]


{'ragas_score': 0.7865, 'answer_relevancy': 0.9341, 'faithfulness': 0.6792}


In [15]:
base_eval = {'ragas_score': 0.8230, 'answer_relevancy': 0.9308, 'faithfulness': 0.7375}

### Leveraging `gpt-4` to improve our `gpt-3.5-turbo` base model!

In [16]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager

finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])

gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
    callback_manager=callback_manager,
)

In [17]:
questions = []
with open("train_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [21]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=gpt_4_context)

query_engine = index.as_query_engine(similarity_top_k=2)

Again, this process will take a few minutes. 

While this is a powerful technique - it is unfortunately quite slow.

In [22]:
for question in questions:
    response = query_engine.query(question)

### Creating the fine-tuning dataset

Now that we have a number of fine-tuning events from our `OpenAIFineTuningHandler()`, let's save them to a `.jsonl` file - the expected format for fine-tuning `gpt-3.5-turbo`!

In [23]:
finetuning_handler.save_finetuning_events("finetuning_events.jsonl")

Wrote 51 examples to finetuning_events.jsonl


In [26]:
import openai
file_response = openai.File.create(file=open("finetuning_events.jsonl", "rb"), purpose='fine-tune')

In [27]:
file_response

<File file id=file-NpALq1Yg4YL2SmFyE2M62xCQ at 0x7f6218bee950> JSON: {
  "object": "file",
  "id": "file-NpALq1Yg4YL2SmFyE2M62xCQ",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 218932,
  "created_at": 1693005279,
  "status": "uploaded",
  "status_details": null
}

In [28]:
import time

response = None

while not response:
  try:
    response = openai.FineTuningJob.create(training_file=file_response.id, model="gpt-3.5-turbo")
  except:
    time.sleep(5)

In [29]:
response

<FineTuningJob fine_tuning.job id=ftjob-y0mYengpv45M6UBlnJohUsiV at 0x7f622d3c7e00> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-y0mYengpv45M6UBlnJohUsiV",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693005307,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-JA8wesYIuJRwJseNuF4F5cpO",
  "result_files": [],
  "status": "created",
  "validation_file": null,
  "training_file": "file-NpALq1Yg4YL2SmFyE2M62xCQ",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [31]:
training_id = response.id

In [32]:
openai.FineTuningJob.retrieve(training_id)

<FineTuningJob fine_tuning.job id=ftjob-y0mYengpv45M6UBlnJohUsiV at 0x7f62183cacc0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-y0mYengpv45M6UBlnJohUsiV",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693005307,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-JA8wesYIuJRwJseNuF4F5cpO",
  "result_files": [],
  "status": "running",
  "validation_file": null,
  "training_file": "file-NpALq1Yg4YL2SmFyE2M62xCQ",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

In [33]:
from IPython.display import clear_output

while openai.FineTuningJob.retrieve(training_id).status == "running":
  clear_output(wait=True)
  time.sleep(5)
  print(openai.FineTuningJob.list_events(id=training_id, limit=10))

print("Done!")

{
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-Vn3flihaJJfPvY9Wpk8gkeaI",
      "created_at": 1693005944,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-ZXInkPIhAGfEX6zuIdGaXBli",
      "created_at": 1693005942,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:handshake::7raTNuPU",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-QT4yN6Ujp1W92tgiEmIzN2ca",
      "created_at": 1693005931,
      "level": "info",
      "message": "Step 150/153: training loss=0.10",
      "data": {
        "step": 150,
        "train_loss": 0.10006184875965118,
        "train_mean_token_accuracy": 0.9444444179534912
      },
      "type": "metrics"
    },
    {
      "object": "fine_tuning

In [34]:
openai.FineTuningJob.retrieve(training_id)

<FineTuningJob fine_tuning.job id=ftjob-y0mYengpv45M6UBlnJohUsiV at 0x7f61723e4b30> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-y0mYengpv45M6UBlnJohUsiV",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1693005307,
  "finished_at": 1693005943,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:handshake::7raTNuPU",
  "organization_id": "org-JA8wesYIuJRwJseNuF4F5cpO",
  "result_files": [
    "file-TdZTCqBHmSFgmueVr7WZ5yKI"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-NpALq1Yg4YL2SmFyE2M62xCQ",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 175374
}

In [35]:
ft_model_id = openai.FineTuningJob.retrieve(training_id).fine_tuned_model

In [36]:
print(ft_model_id)

ft:gpt-3.5-turbo-0613:handshake::7raTNuPU


### Evaluating the fine-tuned model

Now that we've fine-tuned our model on the `gpt-4` enhanced question answers - let's see how it performs on our `raga` evaluation!

In [37]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_id, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [38]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [39]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents, service_context=ft_context)

query_engine = index.as_query_engine(similarity_top_k=2)

In [40]:
contexts = []
answers = []

for question in questions:
    response = query_engine.query(question)
    contexts.append([x.node.get_content() for x in response.source_nodes])
    answers.append(str(response))

In [41]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import answer_relevancy, faithfulness

ds = Dataset.from_dict(
    {
        "question": questions,
        "answer": answers,
        "contexts": contexts,
    }
)

result = evaluate(ds, [answer_relevancy, faithfulness])
print(result)

evaluating with [answer_relevancy]


100%|█████████████████████████████████████████████| 3/3 [01:44<00:00, 34.98s/it]


evaluating with [faithfulness]


100%|█████████████████████████████████████████████| 3/3 [04:18<00:00, 86.30s/it]


{'ragas_score': 0.8811, 'answer_relevancy': 0.9431, 'faithfulness': 0.8267}


In [42]:
ft_eval = {'ragas_score': 0.8599, 'answer_relevancy': 0.9398, 'faithfulness': 0.7925}

### Exploring Differences

Now we can compare the outputs of the two models!

In [43]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

In [44]:
questions = []
with open("eval_questions.txt", "r") as f:
    for line in f:
        questions.append(line.strip())

In [2]:
print(questions[12])

NameError: name 'questions' is not defined

In [46]:
from llama_index.response.notebook_utils import display_response
from llama_index import ServiceContext
from llama_index.llms import OpenAI


gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [47]:
query_engine = index.as_query_engine(service_context=gpt_35_context)

response = query_engine.query(questions[12])

display_response(response)

**`Final Response:`** The bird claimed that reverse engineering enables them to quickly analyze and understand the technology of a spaceship, allowing them to potentially replicate or modify it for their own purposes.

In [48]:
from llama_index import ServiceContext
from llama_index.llms import OpenAI


ft_context = ServiceContext.from_defaults(
    llm=OpenAI(model=ft_model_id, temperature=0.3),
    context_window=2048,  # limit the context window artifically to test refine process
)

In [49]:
query_engine = index.as_query_engine(service_context=ft_context)

response = query_engine.query(questions[12])

display_response(response)

**`Final Response:`** The bird claimed that reverse engineering allows them to bypass the waiting process for a spaceship to pass through their galactic sector in order to assess if it is willing to provide transportation. Through reverse engineering, they can ascertain that a spaceship will offer them a lift and make a request upon its arrival.

In [50]:
metric_list = ["answer_relevancy", "faithfulness"]

for metric in metric_list:
  print("Base model", metric, ":", base_eval[metric])
  print("Fine-tuned model", metric, ":", ft_eval[metric])
  print(f"Improvement {metric} : {(ft_eval[metric] - base_eval[metric])*100:.2f}%")
  print()

Base model answer_relevancy : 0.9308
Fine-tuned model answer_relevancy : 0.9398
Improvement answer_relevancy : 0.90%

Base model faithfulness : 0.7375
Fine-tuned model faithfulness : 0.7925
Improvement faithfulness : 5.50%

